In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install biopython
import numpy as np
import pandas as pd
import sklearn.model_selection
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser

# FASTA to DataFrame of Sequences

In [ ]:
def fastaDF (input_file,gene_n,drug):
  """converts a FASTA file to a pandas DataFrame. 
    parameters:
    input_file is the file path from current directory to the file. should be a .fa or .fasta file (string)
    gene_n is the name of the gene for which the sequences are in the FASTA file (string)
    drug : name of the antibiotic that the species have resistance for (string)
  """

  #import cipro gyrA FASTA and make DataFrame
  with open(input_file) as fasta_file: #cipro
      identifiers = []
      bacteria_IDs = []
      names = []
      lengths = []
      sequences = []
      gene = []
      for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
          identifiers.append(seq_record.id)
          lengths.append(len(seq_record.seq))
          s,d = seq_record.description.split('[')
          d = d[0:len(d)-1]
          bacteria_IDs.append(d)
          bact_name = d.split(' ')[0:2]
          n = ' '.join(bact_name[0:2])
          names.append(n)
          sequences.append(seq_record.seq)
      gene.append(names)
      gene.append(bacteria_IDs)
      gene.append(sequences)
      gene_df = pd.DataFrame(gene,index=['Bacteria Name','Strain',f'{gene_n} Sequence {drug}']).transpose()
  df1=gene_df.set_index('Bacteria Name')
  return df1

In [ ]:
#Cipro gyrA FASTA to DF
gyr_path = '/content/drive/Shareddrives/Project2_Drive/PROJECT 2/combined_sequences/combined_gyra.fasta'
df_gyrA_cipro=fastaDF(gyr_path,'gyrA','C')
df_parC_cipro=fastaDF('/content/drive/Shareddrives/Project2_Drive/PROJECT 2/combined_sequences/combined_parc.fasta','parC','C')
df_gyrA_moxi = fastaDF('/content/drive/Shareddrives/Project2_Drive/PROJECT 2/combined_sequences/gyrA_moxi.fasta','gyrA','M')
df_parC_moxi = fastaDF('/content/drive/Shareddrives/Project2_Drive/PROJECT 2/combined_sequences/parC_moxi.fasta', 'parC', 'M')

In [ ]:
df_cipro = pd.merge(df_gyrA_cipro,df_parC_cipro,how="inner", on=['Bacteria Name','Strain'])
df_cipro = df_cipro.drop("Strain",axis=1)
df_cipro.sort_index()
df_cipro

In [ ]:
df_moxi = pd.merge(df_gyrA_moxi,df_parC_moxi,how="inner", on=['Bacteria Name'],suffixes=(': gyrA moxi',': parC moxi'))
df_moxi = df_moxi.sort_index()
df_moxi=df_moxi.drop_duplicates("Strain: gyrA moxi",keep='first')
df_moxi=df_moxi.drop_duplicates("Strain: parC moxi",keep='first')
df_moxi=df_moxi.drop(["Strain: parC moxi",'Strain: gyrA moxi'],axis=1)
df_moxi.sort_index()
df_moxi

In [ ]:
df = pd.concat([df_moxi,df_cipro],join='outer')
df

# Bag of Words (K-mers)

In [ ]:
def makeKmers(sequ,gene,drug):
  '''
  create list of k-mer words in DataFrame, along with bacteria name as index
  gene is string name
  drug is single letter abbreviation of drug name
  '''

  def getKmers(sequence,ksize=6):
    '''
    function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
    sequence input must be in String format
    '''
    return [sequence[x:x+ksize] for x in range(len(sequence) - ksize + 1)]

  ind1 = []
  seq1 = []
  df_test = []
  for i, s in sequ.iterrows():
      ind1.append(i)
      seq1.append(str(s[f'{gene} Sequence {drug}']))
  df_test.append(ind1)
  df_test.append(seq1)
  df_test = pd.DataFrame(df_test,index=['Bacteria Name',f'{gene}-{drug} Seq']).transpose()
  df_test=df_test.set_index(['Bacteria Name'])
  df_test=df_test.sort_index()
  df=df_test
  df[f'{gene}-{drug}']=df.apply(lambda x: getKmers(x[f'{gene}-{drug} Seq']), axis=1)
  df = df.drop([f'{gene}-{drug} Seq'],axis=1)
  return df

In [ ]:
#k-mer DF for cipro genes
df_1 = makeKmers(df_cipro,'gyrA','C')
df_2 = makeKmers(df_cipro,'parC','C')
df_words = df_1.merge(df_2,on='Bacteria Name')
df_words

In [ ]:
#k-mer DF for cipro genes
df_1 = makeKmers(df_moxi,'gyrA','M')
df_2 = makeKmers(df_moxi,'parC','M')
df_words_m = df_1.merge(df_2,on='Bacteria Name')
df_words_m

In [ ]:
df_words_m.to_csv('/content/drive/Shareddrives/Project2_Drive/PROJECT 2/df_m.csv')

In [ ]:
gyrA_C_word = list(df_words['gyrA-C'])
for item in range(len(gyrA_C_word)):
    gyrA_C_word[item] = ' '.join(gyrA_C_word[item])
C_spec = df_words.index.values                         #C_ for C species
parC_C_word = list(df_words['parC-C'])
for item in range(len(parC_C_word)):
    parC_C_word[item] = ' '.join(parC_C_word[item])

In [ ]:
gyrA_M_word = list(df_words_m['gyrA-M'])
for item in range(len(gyrA_M_word)):
    gyrA_M_word[item] = ' '.join(gyrA_M_word[item])
M_spec = df_words_m.index.values                         #M_ for moxi species
parC_M_word = list(df_words_m['parC-M'])
for item in range(len(parC_M_word)):
    parC_M_word[item] = ' '.join(parC_M_word[item])

In [ ]:
spec=np.append(C_spec,M_spec)
gyrA_words=gyrA_C_word+gyrA_M_word
parC_words=parC_C_word+parC_M_word

In [ ]:
'''
 Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
gAC = cv.fit_transform(human_texts)
pCC = cv.transform(chimp_texts)
gAM = cv.transform(dog_texts)
'''

In [ ]:
df_kmer = pd.concat([df_words,df_words_m],join='outer')
df_kmer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4)) #The n-gram size of 4 is previously determined by testing
X_gyrA = cv.fit_transform(gyrA_words)
X_parC = cv.transform(parC_words)
print(X_gyrA.shape)
print(X_parC.shape)

# MIC DataFrame (Y values)

In [ ]:
weighted_MIC = pd.read_csv("/content/drive/Shareddrives/Project2_Drive/PROJECT 2/MIC_resistance_all.csv")
weighted_MIC = weighted_MIC.set_index("Bacteria Name")
weighted_MIC.dropna(axis=0,how="all",inplace=True)
weighted_MIC=weighted_MIC.drop('Staphylococcus hominis',axis=0)
weighted_MIC

In [ ]:
MIC_df_cipro = weighted_MIC[["Ciprofloxacin Weighted MIC"]]
MIC_df_cipro = MIC_df_cipro.dropna(how='any')
MIC_df_cipro = MIC_df_cipro.rename(columns={"Ciprofloxacin Weighted MIC":"cipro MIC"})
MIC_df_cipro
y_cipro = MIC_df_cipro.iloc[:, 0].values # y_cipro for Cipro MIC
len(y_cipro)

In [ ]:
MIC_df_moxi = weighted_MIC[["Moxifloxacin Weighted MIC"]]
MIC_df_moxi = MIC_df_moxi.dropna(how='any')
MIC_df_moxi = MIC_df_moxi.rename(columns={"Moxifloxacin Weighted MIC":"moxi MIC"})
len(MIC_df_moxi)
#y_moxi = MIC_df_moxi.iloc[:, 0].values # y_moxi for moxi MIC
#y_moxi

In [ ]:
y_MIC=np.append(y_cipro,y_moxi)

In [ ]:
MIC_df = pd.concat([MIC_df_cipro,MIC_df_moxi],join='outer')
MIC_df

# Percent Identity values

In [ ]:
PIM_df_cipro = pd.read_csv("/content/drive/Shareddrives/Project2_Drive/PROJECT 2/ciprofloaxin_PIM_singleval.csv")
PIM_df_cipro = PIM_df_cipro.set_index("Bacteria Name")
PIM_df_cipro = PIM_df_cipro.sort_index()
PIM_df_cipro = PIM_df_cipro.rename(columns={"gyrA":"c-gyrA", "parC": "c-parC"})
PIM_df_cipro

In [ ]:
PIM_df_moxi = pd.read_csv("/content/drive/Shareddrives/Project2_Drive/PROJECT 2/moxi_PIM.csv")
PIM_df_moxi = PIM_df_moxi.set_index("Bacteria Name")
PIM_df_moxi = PIM_df_moxi.sort_index()
PIM_df_moxi = PIM_df_moxi.rename(columns={"gyrA PIM":"m-gyrA", "parC PIM": "m-parC"})
PIM_df_moxi

In [ ]:
PIM_df=pd.concat([PIM_df_cipro,PIM_df_moxi],join='outer',sort=True)
PIM_df.groupby(level=0).sum()
PIM_df

# Combining all data into DF

In [ ]:
cipro_df = pd.merge(MIC_df_cipro,PIM_df_cipro,how="inner",on=["Bacteria Name"])
cipro_df.drop_duplicates()
cipro_df2=pd.merge(cipro_df,df_words,how="inner",on=["Bacteria Name"])
cipro_df2.drop_duplicates(subset='cipro MIC')
cipro_df2 #ALL CIPRO DATA

In [ ]:
moxi_df = pd.merge(MIC_df_moxi,PIM_df_moxi,how="inner",on=["Bacteria Name"])
moxi_df.groupby(level=0)
moxi_df=moxi_df.drop_duplicates()
moxi_df

In [ ]:
moxi_df2 = pd.merge(moxi_df,df_words_m,how="inner",on=["Bacteria Name"])
moxi_df2.drop_duplicates(subset='moxi MIC') #ALL MOXI DATA


```
# df = pd.merge(df,MIC_df_cipro,how="inner", on=['Bacteria Name'])
df
gyrA_PIM = pd.read_csv("/content/drive/Shareddrives/Project2_Drive/Project 1/gyrA_PIM.csv")
gyrA_PIM = gyrA_PIM.set_index("Bacteria Name")
gyrA_PIM
df2 = pd.merge(gyrA_PIM,df,how="inner", on=['Bacteria Name','Strain'])
df2
df_rf = df2.drop(["Strain","Sequence: gyrA","Sequence: parC"],axis=1)
df_rf
```



# Data Separation for ML

In [ ]:
MIC_val_cipro = np.array(cipro_df['cipro MIC'])
columns_list = list(cipro_df.columns)
species_list = list(cipro_df.index)
cipro_df = cipro_df.drop("cipro MIC", axis=1)
cipro_df

In [ ]:
cipro_df = np.array(cipro_df)
cipro_df

# Random Forest Regression K-mer

In [ ]:
train_features, test_features, train_labels, test_labels = sklearn.model_selection.train_test_split(cipro_df, MIC_val_cipro, test_size = 0.4,random_state=38,shuffle=False)

In [ ]:
test_labels

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 18)
# Train the model on training data
rf.fit(train_features, train_labels)

In [ ]:
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
print(predictions)
print(test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# Model Comparison

In [ ]:
# Imports
import plotly.graph_objects as go

# Grab the data
bacteria_names = cipro_df2.index.array
model1_predictions = predictions
model2_predictions = rf.predict(test_features)
actual_values = test_labels

# Testing
print(len(bacteria_names))
print(len(model1_predictions))
print(len(model2_predictions))
print(len(actual_values))

# Plot the chart
fig = go.Figure(
  data=[
    go.Bar(name='Model 1 Predicted MIC', x=bacteria_names, y=model1_predictions),
    go.Bar(name='Model 2 Predicted MIC', x=bacteria_names, y=model2_predictions),
    go.Bar(name='Actual MIC', x=bacteria_names, y=actual_values)
  ],
  layout={
    'yaxis': {'title': 'Predicted MIC Value'},
    'xaxis': {'title': 'Bacteria Name'},
    'title': 'Bacteria Names vs. Predicted MIC Value'
  }
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()